In [1]:
import os
from docx import Document
from docx.shared import Inches
from datetime import datetime
import uuid

# Bases de datos

In [2]:
DOCUMENTOS_PATH = "documentos/"
FIRMAS_PATH = "firmas/"
MACHOTES_PATH = "machotes/"

In [3]:
base_usuarios = {
    "ana.lopez@casamonarca.org": "Ana López",
    "carlos.ramirez@casamonarca.org": "Carlos Ramírez",
    "maria.gonzalez@casamonarca.org": "María González"
}
# Formato de firmas firmas/maria_dot_gonzalez_at_casamonarca_dot_org.png


# Codigo

In [4]:
def normalizar_correo(correo):
    return correo.strip().lower().replace("@", "_at_").replace(".", "_dot_")

In [5]:
# En local
def buscar_firma_por_correo(correo):
    archivo = normalizar_correo(correo) + ".png"
    path = os.path.join(FIRMAS_PATH, archivo)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Firma no encontrada para {correo}")
    return path
# En produccion base online

# En local
def buscar_nombre_por_correo(correo, base_usuarios):
    return base_usuarios.get(correo, None)

# En produccion pasarlo como parametro o buscarlo en azure

# En local
def obtener_documento_por_id(doc_id):
    path = os.path.join(DOCUMENTOS_PATH, f"doc_{doc_id}.docx")
    if not os.path.exists(path):
        raise FileNotFoundError(f"Documento con ID {doc_id} no encontrado")
    return path

# En produccion pasarlo como parametro o buscarlo en azure




In [6]:
def insertar_firma_y_nombre_por_id(doc_id, correo, base_usuarios, placeholder_nombre, rol):
    """
    Inserta la firma y nombre en un documento buscando por ID, y guarda sobre el mismo archivo.
    """
    doc_path = obtener_documento_por_id(doc_id)
    doc = Document(doc_path)

    nombre = base_usuarios.get(correo)
    if not nombre:
        raise ValueError(f"Nombre no encontrado para {correo}")

    firma_path = buscar_firma_por_correo(correo)

    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                for i, paragraph in enumerate(cell.paragraphs):
                    if placeholder_nombre in paragraph.text:
                        # Insertar firma
                        parrafo_firma = cell.paragraphs[i].insert_paragraph_before()
                        run = parrafo_firma.add_run()
                        run.add_picture(firma_path, width=Inches(2))
                        
                        # Insertar una línea de firma si quieres que siempre esté
                        parrafo_linea = paragraph.insert_paragraph_before()
                        parrafo_linea.add_run("____________________")

                        # Reemplazar marcador por nombre + rol
                        paragraph.text = f"{rol}: {nombre}\n(Personal autorizado)"


    # Guardar editando sobre el mismo documento
    doc.save(doc_path)

In [7]:
insertar_firma_y_nombre_por_id(
    doc_id="6993f3dc-f699-4cf0-a598-7f908a973257",
    correo="carlos.ramirez@casamonarca.org",
    base_usuarios=base_usuarios,
    placeholder_nombre="{nombre_inventario}",
    rol="Aprobado por")